In [116]:
# 1. Imports
import numpy as np
import pandas as pd
import pymc as pm
import pytensor.tensor as pt
import matplotlib.pyplot as plt
import arviz as az
from scipy.stats import mode

In [117]:
# 2. Load and Trim Data from 2008 onward
df = pd.read_csv("../../data/raw/BrentOilPrices.csv", parse_dates=["Date"])
df = df.dropna(subset=["Price"])
df = df.sort_values("Date")

# Only keep data from 2008 onward
df = df[df["Date"] >= "2008-01-01"].copy()

# Extract values
prices = df["Price"].values
dates = df["Date"].values
n = len(prices)

/tmp/ipykernel_59877/2121933993.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv("../../data/raw/BrentOilPrices.csv", parse_dates=["Date"])


In [118]:
# 3. Number of Change Points
K = 8  # You can increase this to more if needed

In [119]:
# 4. Model
with pm.Model() as model:
    taus_unordered = pm.DiscreteUniform('taus_unordered', lower=0, upper=n-1, shape=K)
    ordered_taus = pm.Deterministic('ordered_taus', pt.sort(taus_unordered))

    mus = pm.Normal('mus', mu=np.mean(prices), sigma=20, shape=K + 1)
    sigmas = pm.HalfNormal('sigmas', sigma=10, shape=K + 1)

    idx = pt.arange(n)
    mu_vals = pt.zeros(n)
    sigma_vals = pt.zeros(n)

    start = 0
    for i in range(K):
        end = ordered_taus[i]
        mask = (idx >= start) & (idx < end)
        mu_vals = pt.set_subtensor(mu_vals[mask], mus[i])
        sigma_vals = pt.set_subtensor(sigma_vals[mask], sigmas[i])
        start = end

    mu_vals = pt.set_subtensor(mu_vals[idx >= start], mus[K])
    sigma_vals = pt.set_subtensor(sigma_vals[idx >= start], sigmas[K])

    y_obs = pm.Normal('y_obs', mu=mu_vals, sigma=sigma_vals, observed=prices)

In [120]:
# 5. Sampling
with model:
    trace = pm.sample(1000, tune=1000, target_accept=0.95, return_inferencedata=True)

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [taus_unordered]
>NUTS: [mus, sigmas]


/home/teshager/Documents/projects/10Academy/brent-oil-changepoint-analysis/venv/lib/python3.12/site-packages/rich/l
ive.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 158 seconds.
There were 1950 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


In [121]:
# 6. Extract change points
taus_samples = trace.posterior['ordered_taus'].values.reshape(-1, K)
mode_taus, _ = mode(taus_samples, axis=0, keepdims=False)
unique_taus = np.unique(mode_taus)

In [122]:
# 7. Print detected change points
print("🔍 All detected change point dates:")
for tau in unique_taus:
    print(pd.to_datetime(dates[tau]))

🔍 All detected change point dates:
2014-12-08 00:00:00
2015-04-16 00:00:00
2015-06-15 00:00:00
2015-07-06 00:00:00
2015-08-03 00:00:00
2015-08-21 00:00:00


In [123]:
# Append start (0) and end (n) to get full segments
full_taus = np.concatenate([[0], np.sort(mode_taus), [n]])

# Calculate segment means
segment_means = [prices[full_taus[i]:full_taus[i+1]].mean() for i in range(len(full_taus)-1)]

# Calculate mean differences
significant_taus = []
for i in range(1, len(full_taus)-1):
    mean_before = segment_means[i-1]
    mean_after = segment_means[i]
    delta = abs(mean_after - mean_before)

    if delta >= 5:  # adjust threshold as needed
        significant_taus.append(full_taus[i])

# Save only significant change points
change_points = [
    {
        "change_point_index": int(tau),
        "change_point_date": str(pd.to_datetime(dates[tau]).date()),
        "event": "Auto-detected (significant)",
        "description": f"ΔMean ≈ {round(abs(segment_means[i] - segment_means[i-1]), 2)}"
    }
    for i, tau in enumerate(significant_taus)
]

change_points_df = pd.DataFrame(change_points)
change_points_df.to_csv("../../data/processed/change_points.csv", index=False)

print("✅ Significant change points saved.")


✅ Significant change points saved.


/tmp/ipykernel_59877/204905963.py:5: RuntimeWarning: Mean of empty slice.
  segment_means = [prices[full_taus[i]:full_taus[i+1]].mean() for i in range(len(full_taus)-1)]
/home/teshager/Documents/projects/10Academy/brent-oil-changepoint-analysis/venv/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [124]:
change_points_df

,change_point_index,change_point_date,event,description
0,1743,2014-12-08,Auto-detected (significant),ΔMean ≈ 33.32
1,1832,2015-04-16,Auto-detected (significant),ΔMean ≈ 40.33
2,1908,2015-08-03,Auto-detected (significant),ΔMean ≈ 7.67
3,1922,2015-08-21,Auto-detected (significant),ΔMean ≈ 2.53


In [125]:
print("🔍 All detected change point dates:")
for tau in unique_taus:
    print(pd.to_datetime(dates[tau]))


🔍 All detected change point dates:
2014-12-08 00:00:00
2015-04-16 00:00:00
2015-06-15 00:00:00
2015-07-06 00:00:00
2015-08-03 00:00:00
2015-08-21 00:00:00
